# Import libraries

In [ ]:
# Import libraries
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model  
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

# Importing libraries
from tensorflow import keras
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import filedialog
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input

# Model

In [ ]:
model_vgg = VGG16()
model_vgg = Model(inputs=model_vgg.inputs, outputs=model_vgg.layers[-2].output)

In [ ]:
from keras.models import load_model
model_final = load_model('model_50.h5')

# Definitions

In [ ]:
max_length = 35

In [ ]:
# Load tokenizer from pickle
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [ ]:
def create_feature(image):
  # Convert image pixels to numpy array
  image = img_to_array(image)
  # Reshape data for model
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  # Preprocess image for vgg
  image = preprocess_input(image)
  # Extract features
  feature = model_vgg.predict(image, verbose=0)
  # Store feature
  return feature

In [ ]:
def idx_to_word(integer, tokenizer):
  for word, index in tokenizer.word_index.items():
      if index == integer:
        return word
  return None

In [ ]:
# Generate caption for an image (using final model)
def predict_caption(model, image, tokenizer, max_length):
  # Add start tag for generation process
  in_text = 'startseq'
  # Iterate over the max length of sequence
  for i in range(max_length):
    # Encode input sequence
    sequence = tokenizer.texts_to_sequences([in_text])[0]
    # Pad the sequence
    sequence = pad_sequences([sequence], max_length)
    # Predict next word
    yhat = model.predict([image, sequence], verbose=0)
    # Get index with high probability
    yhat = np.argmax(yhat)
    # Convert index to word
    word = idx_to_word(yhat, tokenizer)
    # Stop if word not found
    if word is None:
      break
    # Sppend word as input for generating next word
    in_text += " " + word
#     Stop if we reach end tag
    if word == 'endseq':
      break
  return in_text

In [ ]:
def upload_img():
    """
    A function to upload the image from the local computer to the application for 
    interaction, we also reframe and adjust the picture with suitable size. 
    """
    global img, image_data
    for img_display in frame.winfo_children():
        img_display.destroy()

    image_data = filedialog.askopenfilename(initialdir="/", title="Choose an image",
                                            filetypes=(("all files", "*.*"), ("png files", "*.png"), ("jpg files", "*.jpg")))
    basewidth = 300
    img = Image.open(image_data)
    wpercent = (basewidth / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((basewidth, hsize), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    file_name = image_data.split('/')
    #panel = tk.Label(frame, text=str(file_name[len(file_name) - 1]).upper()).pack()
    panel_image = tk.Label(frame, image=img).pack()

In [ ]:
def captioning():
    """
    A function where it uses the model to generate caption matching with the 
    new picture, and stores the caption for output.
    """
    original = Image.open(image_data)
    original = original.resize((224, 224), Image.ANTIALIAS)
    caption = predict_caption(model_final, create_feature(original), tokenizer, max_length)
    half_len = int(len(caption)//2)
    space_index = caption.find(" ", half_len)
    caption1 = str(caption[9].upper() + caption[10:space_index])
    caption2 = str(caption[space_index+1:-7] + ".")
    table1 = tk.Label(frame, text=(caption1), font=("Helvetica", 12)).pack()
    table2 = tk.Label(frame, text=(caption2), font=("Helvetica", 12)).pack()

# GUI

In [ ]:
root = tk.Tk()
root.title('IMAGE CAPTION GENERATOR')
# root.iconbitmap('icon.ico')
root.resizable(False, False)
tit = tk.Label(root, text="IMAGE CAPTION GENERATOR", padx=25, pady=6, font=("Helvetica", 14)).pack()
canvas = tk.Canvas(root, height=550, width=600, bg='#EA4630')
canvas.pack()
frame = tk.Frame(root, bg='#146B3A')
frame.place(relwidth=0.8, relheight=0.8, relx=0.1, rely=0.1)
chose_image = tk.Button(root, text='Choose Image',
                        padx=35, pady=15, font=("Helvetica", 12),
                        fg="black", bg="#F8B229", command=upload_img, activebackground="#add8e6")
chose_image.pack(side=tk.LEFT)

caption_image = tk.Button(root, text='Classify Image',
                          padx=35, pady=15, font=("Helvetica", 12),
                          fg="black", bg="#F8B229", command=captioning, activebackground="#add8e6")
caption_image.pack(side=tk.RIGHT)
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\TDC\New folder\lib\site-packages\PIL\Image.py", line 2916, in open
    fp.seek(0)
AttributeError: 'str' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\TDC\New folder\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-21-78fcc81aa499>", line 9, in upload_img
    img = Image.open(image_data)
  File "D:\TDC\New folder\lib\site-packages\PIL\Image.py", line 2918, in open
    fp = io.BytesIO(fp.read())
AttributeError: 'str' object has no attribute 'read'
Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\TDC\New folder\lib\site-packages\PIL\Image.py", line 2916, in open
    fp.seek(0)
AttributeError: 'str' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File